In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import numpy as np

The function normalize is taken from [Prabhanshu Attri et al](https://keras.io/examples/timeseries/timeseries_weather_forecasting/) 

In [11]:
def normalize(data, train_split):
    data_mean = np.nanmean(data[:train_split], axis=0)
    data_std = np.nanstd(data[:train_split], axis=0)
    return (data - data_mean) / data_std

Observations are recorded every hour. 
We are tracking data from past 720 timestamps (30 days) or 336 timestamps (2 weeks). This data will be used to predict the rainfall amount after 24 timestamps (1 day).
All the following functions are used to fit a specific neural network to the data. These functions take as input the data and the number of outputs of 1 or more layer. The other parameters have a default values but they can be changed.

In [12]:
def prediction_1LSTM(data, a, split_fraction=0.715, step=1, past=720, future=24, learning_rate=0.001, batch_size=256, epochs=10):
    train_split = int(split_fraction * int(data.shape[0]))
    train_data = data.loc[0 : train_split - 1]
    val_data = data.loc[train_split:]
    start = past + future #start of y lablels
    end = start + train_split #end of y labels
    x_end = len(val_data) - past - future#index of the last x to consider for the validation set

    x_train = train_data.drop('rain', axis=1)
    y_train = data.iloc[start:end,0]

    x_val = val_data.drop('rain', axis=1).iloc[:x_end]
    y_val = val_data.iloc[start:,0]

    sequence_length = int(past/step)
    dataset_train = keras.preprocessing.timeseries_dataset_from_array(
    x_train,#x
    y_train,#y
    sequence_length=sequence_length,#number of x needed to predict the y
    sampling_rate=step,#how many timestamp to skip
    batch_size=batch_size,#number of timeseries in each batch
    )

    dataset_val = keras.preprocessing.timeseries_dataset_from_array(
        x_val,
        y_val,
        sequence_length=sequence_length,
        sampling_rate=step,
        batch_size=batch_size,
    )
    for batch in dataset_train.take(1):
        inputs, targets = batch
        
    #NN
    inputs = keras.layers.Input(shape=(inputs.shape[1], inputs.shape[2]))
    lstm_out = keras.layers.LSTM(a)(inputs)#long short term memory layer
    outputs = keras.layers.Dense(1)(lstm_out)

    model = keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), loss="mse")#Adam for gradient descent
    print(model.summary())
    
    es_callback = keras.callbacks.EarlyStopping(monitor="val_loss", min_delta=0, patience=5)

    modelckpt_callback = keras.callbacks.ModelCheckpoint( #allows to save model parameters for each epoch such that the best 
        #can be found (no risk of losing efficiency in successive epochs)
        monitor="val_loss",
        filepath="model_checkpoint.h5",
        verbose=1,
        save_weights_only=True,
        save_best_only=True,
    )

    history = model.fit(
        dataset_train,
        epochs=epochs,
        validation_data=dataset_val,
        callbacks=[es_callback, modelckpt_callback],
    )


In [20]:
def prediction_1LSTM_2D(data, a, b, split_fraction=0.715, step=1, past=720, future=24, learning_rate=0.001, batch_size=256, epochs=10):
    train_split = int(split_fraction * int(data.shape[0]))
    train_data = data.loc[0 : train_split - 1]
    val_data = data.loc[train_split:]
    start = past + future #start of y lablels
    end = start + train_split #end of y labels
    x_end = len(val_data) - past - future#index of the last x to consider for the validation set

    x_train = train_data.drop('rain', axis=1)
    y_train = data.iloc[start:end,0]

    x_val = val_data.drop('rain', axis=1).iloc[:x_end]
    y_val = val_data.iloc[start:,0]

    sequence_length = int(past/step)
    dataset_train = keras.preprocessing.timeseries_dataset_from_array(
    x_train,#x
    y_train,#y
    sequence_length=sequence_length,#number of x neede to predict the y
    sampling_rate=step,#how many timestamp to skip
    batch_size=batch_size,#number of timeseries in each batch
    )

    dataset_val = keras.preprocessing.timeseries_dataset_from_array(
        x_val,
        y_val,
        sequence_length=sequence_length,
        sampling_rate=step,
        batch_size=batch_size,
    )
    for batch in dataset_train.take(1):
        inputs, targets = batch
        
    #NN
    inputs = keras.layers.Input(shape=(inputs.shape[1], inputs.shape[2]))
    lstm_out = keras.layers.LSTM(a)(inputs)#long short term memory layer
    dense_out = keras.layers.Dense(b)(lstm_out)
    outputs = keras.layers.Dense(1)(dense_out)

    model = keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), loss="mse")#Adam for gradient descent
    print(model.summary())
    
    es_callback = keras.callbacks.EarlyStopping(monitor="val_loss", min_delta=0, patience=5)

    modelckpt_callback = keras.callbacks.ModelCheckpoint( #allows to save model parameters for each epoch such that the best 
        #can be found (no risk of losing efficiency in successive epochs)
        monitor="val_loss",
        filepath="model_checkpoint.h5",
        verbose=1,
        save_weights_only=True,
        save_best_only=True,
    )

    history = model.fit(
        dataset_train,
        epochs=epochs,
        validation_data=dataset_val,
        callbacks=[es_callback, modelckpt_callback],
    )

In [15]:
def prediction_DENSE(data, split_fraction=0.715, step=1, past=720, future=24, learning_rate=0.001, batch_size=256, epochs=10):
    train_split = int(split_fraction * int(data.shape[0]))
    train_data = data.loc[0 : train_split - 1]
    val_data = data.loc[train_split:]
    start = past + future #start of y lablels
    end = start + train_split #end of y labels
    x_end = len(val_data) - past - future#index of the last x to consider for the validation set

    x_train = train_data.drop('rain', axis=1)
    y_train = data.iloc[start:end,0]

    x_val = val_data.drop('rain', axis=1).iloc[:x_end]
    y_val = val_data.iloc[start:,0]

    sequence_length = int(past/step)
    dataset_train = keras.preprocessing.timeseries_dataset_from_array(
    x_train,#x
    y_train,#y
    sequence_length=sequence_length,#number of x neede to predict the y
    sampling_rate=step,#how many timestamp to skip
    batch_size=batch_size,#number of timeseries in each batch
    )

    dataset_val = keras.preprocessing.timeseries_dataset_from_array(
        x_val,
        y_val,
        sequence_length=sequence_length,
        sampling_rate=step,
        batch_size=batch_size,
    )
    for batch in dataset_train.take(1):
        inputs, targets = batch
        
    #NN  
    inputs = keras.layers.Input(shape=(inputs.shape[1], inputs.shape[2]))
    dense1 = keras.layers.Dense(50, activation="relu")(inputs)
    dense2 = keras.layers.Dense(20, activation="relu")(dense1)
    dense3 = keras.layers.Dense(5, activation="relu")(dense2)
    outputs = keras.layers.Dense(1, activation="sigmoid")(dense3)


    model = keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), loss="mse")#Adam for gradient descent
    print(model.summary())
    
    es_callback = keras.callbacks.EarlyStopping(monitor="val_loss", min_delta=0, patience=5)

    modelckpt_callback = keras.callbacks.ModelCheckpoint( #allows to save model parameters for each epoch such that the best 
        #can be found (no risk of losing efficiency in successive epochs)
        monitor="val_loss",
        filepath="model_checkpoint.h5",
        verbose=1,
        save_weights_only=True,
        save_best_only=True,
    )

    history = model.fit(
        dataset_train,
        epochs=epochs,
        validation_data=dataset_val,
        callbacks=[es_callback, modelckpt_callback],
    )

In [16]:
def prediction_1GRU(data, a, split_fraction=0.715, step=1, past=720, future=24, learning_rate=0.001, batch_size=256, epochs=10):
    train_split = int(split_fraction * int(data.shape[0]))
    train_data = data.loc[0 : train_split - 1]
    val_data = data.loc[train_split:]
    start = past + future #start of y lablels
    end = start + train_split #end of y labels
    x_end = len(val_data) - past - future#index of the last x to consider for the validation set

    x_train = train_data.drop('rain', axis=1)
    y_train = data.iloc[start:end,0]

    x_val = val_data.drop('rain', axis=1).iloc[:x_end]
    y_val = val_data.iloc[start:,0]

    sequence_length = int(past/step)
    dataset_train = keras.preprocessing.timeseries_dataset_from_array(
    x_train,#x
    y_train,#y
    sequence_length=sequence_length,#number of x neede to predict the y
    sampling_rate=step,#how many timestamp to skip
    batch_size=batch_size,#number of timeseries in each batch
    )

    dataset_val = keras.preprocessing.timeseries_dataset_from_array(
        x_val,
        y_val,
        sequence_length=sequence_length,
        sampling_rate=step,
        batch_size=batch_size,
    )
    for batch in dataset_train.take(1):
        inputs, targets = batch
        
    #NN
    inputs = keras.layers.Input(shape=(inputs.shape[1], inputs.shape[2]))
    lstm_out = keras.layers.GRU(a)(inputs)#gru layer
    outputs = keras.layers.Dense(1)(lstm_out)

    model = keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), loss="mse")#Adam for gradient descent
    print(model.summary())
    
    es_callback = keras.callbacks.EarlyStopping(monitor="val_loss", min_delta=0, patience=5)

    modelckpt_callback = keras.callbacks.ModelCheckpoint( #allows to save model parameters for each epoch such that the best 
        #can be found (no risk of losing efficiency in successive epochs)
        monitor="val_loss",
        filepath="model_checkpoint.h5",
        verbose=1,
        save_weights_only=True,
        save_best_only=True,
    )

    history = model.fit(
        dataset_train,
        epochs=epochs,
        validation_data=dataset_val,
        callbacks=[es_callback, modelckpt_callback],
    )

In [17]:
def prediction_1GRU_2D(data, a, b, split_fraction=0.715, step=1, past=720, future=24, learning_rate=0.001, batch_size=256, epochs=10):
    train_split = int(split_fraction * int(data.shape[0]))
    train_data = data.loc[0 : train_split - 1]
    val_data = data.loc[train_split:]
    start = past + future #start of y lablels
    end = start + train_split #end of y labels
    x_end = len(val_data) - past - future#index of the last x to consider for the validation set

    x_train = train_data.drop('rain', axis=1)
    y_train = data.iloc[start:end,0]

    x_val = val_data.drop('rain', axis=1).iloc[:x_end]
    y_val = val_data.iloc[start:,0]

    sequence_length = int(past/step)
    dataset_train = keras.preprocessing.timeseries_dataset_from_array(
    x_train,#x
    y_train,#y
    sequence_length=sequence_length,#number of x neede to predict the y
    sampling_rate=step,#how many timestamp to skip
    batch_size=batch_size,#number of timeseries in each batch
    )

    dataset_val = keras.preprocessing.timeseries_dataset_from_array(
        x_val,
        y_val,
        sequence_length=sequence_length,
        sampling_rate=step,
        batch_size=batch_size,
    )
    for batch in dataset_train.take(1):
        inputs, targets = batch
        
    #NN
    inputs = keras.layers.Input(shape=(inputs.shape[1], inputs.shape[2]))
    lstm_out = keras.layers.GRU(a)(inputs)
    dense_out = keras.layers.Dense(b)(lstm_out)
    outputs = keras.layers.Dense(1)(dense_out)

    model = keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), loss="mse")#Adam for gradient descent
    print(model.summary())
    
    es_callback = keras.callbacks.EarlyStopping(monitor="val_loss", min_delta=0, patience=5)

    modelckpt_callback = keras.callbacks.ModelCheckpoint( #allows to save model parameters for each epoch such that the best 
        #can be found (no risk of losing efficiency in successive epochs)
        monitor="val_loss",
        filepath="model_checkpoint.h5",
        verbose=1,
        save_weights_only=True,
        save_best_only=True,
    )

    history = model.fit(
        dataset_train,
        epochs=epochs,
        validation_data=dataset_val,
        callbacks=[es_callback, modelckpt_callback],
    )

In [40]:
def prediction_1GRU_1LSTM_1DENSE(data, a, b, c, split_fraction=0.715, step=1, past=720, future=24, learning_rate=0.001, batch_size=256, epochs=10):
    train_split = int(split_fraction * int(data.shape[0]))
    train_data = data.loc[0 : train_split - 1]
    val_data = data.loc[train_split:]
    start = past + future #start of y lablels
    end = start + train_split #end of y labels
    x_end = len(val_data) - past - future#index of the last x to consider for the validation set

    x_train = train_data.drop('rain', axis=1)
    y_train = data.iloc[start:end,0]

    x_val = val_data.drop('rain', axis=1).iloc[:x_end]
    y_val = val_data.iloc[start:,0]

    sequence_length = int(past/step)
    dataset_train = keras.preprocessing.timeseries_dataset_from_array(
    x_train,#x
    y_train,#y
    sequence_length=sequence_length,#number of x neede to predict the y
    sampling_rate=step,#how many timestamp to skip
    batch_size=batch_size,#number of timeseries in each batch
    )

    dataset_val = keras.preprocessing.timeseries_dataset_from_array(
        x_val,
        y_val,
        sequence_length=sequence_length,
        sampling_rate=step,
        batch_size=batch_size,
    )
    for batch in dataset_train.take(1):
        inputs, targets = batch
        
    #NN
    inputs = keras.layers.Input(shape=(inputs.shape[1], inputs.shape[2]))
    gru_out = keras.layers.GRU(a, return_sequences=True)(inputs)
    lstm_out = keras.layers.LSTM(b)(gru_out)
    dense_out = keras.layers.Dense(c)(lstm_out)
    outputs = keras.layers.Dense(1)(dense_out)

    model = keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), loss="mse")#Adam for gradient descent
    print(model.summary())
    
    es_callback = keras.callbacks.EarlyStopping(monitor="val_loss", min_delta=0, patience=5)

    modelckpt_callback = keras.callbacks.ModelCheckpoint( #allows to save model parameters for each epoch such that the best 
        #can be found (no risk of losing efficiency in successive epochs)
        monitor="val_loss",
        filepath="model_checkpoint.h5",
        verbose=1,
        save_weights_only=True,
        save_best_only=True,
    )

    history = model.fit(
        dataset_train,
        epochs=epochs,
        validation_data=dataset_val,
        callbacks=[es_callback, modelckpt_callback],
    )